In [2]:
import pandas as pd
import numpy as np

from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras import regularizers
import tensorflow as tf

from sklearn.preprocessing import MinMaxScaler

In [4]:
train_data = pd.read_csv("../train_data.csv")
dev_data = pd.read_csv("../dev_data.csv")

train_data = train_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
train_data["is_retweet"] = 1*train_data["is_retweet"]
train_data["is_modified_retweet"] = 1*train_data["is_modified_retweet"]

dev_data = dev_data.drop(["Unnamed: 0"], axis=1)
# Change True false into 1 and 0
dev_data["is_retweet"] = 1*dev_data["is_retweet"]
dev_data["is_modified_retweet"] = 1*dev_data["is_modified_retweet"]

In [5]:
sc = train_data.groupby("author_Id").count().sort_values("tweet",ascending = False)#.head()
set_y = list(sc.index)
def id_to_int_cat(id_ind):
    return np.argwhere(train_data["author_Id"].values[id_ind]==set_y)[0][0]
temp = [id_to_int_cat(i) for i in range(len(train_data["author_Id"]))]
y_train = to_categorical(temp,dtype=int)
X_train = train_data.drop(['tweet','author_Id'],axis=1).values
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
y_train = to_categorical(temp,dtype=int)

In [20]:
filepath = "modeln.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
early_stop =  EarlyStopping(monitor='val_loss',patience=3, min_delta=0.0001)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logn")
callbacks_list = [checkpoint,tensorboard_callback]#,early_stop,]


epochs =1000
batch_size = 4096



In [21]:
new_model = load_model(filepath)

history = new_model.fit(X_train, y_train, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_split=0.25,
                    callbacks=callbacks_list)

Train on 185024 samples, validate on 61675 samples
Epoch 1/1000
184320/185024 [============================>.] - ETA: 0s - loss: 7.5196 - accuracy: 0.0235
Epoch 00001: val_loss improved from inf to 8.17522, saving model to modeln.h5
185024/185024 [==============================] - 156s 843us/sample - loss: 7.5201 - accuracy: 0.0235 - val_loss: 8.1752 - val_accuracy: 0.0234
Epoch 2/1000
 45056/185024 [======>.......................] - ETA: 1:08 - loss: 7.4848 - accuracy: 0.0238

KeyboardInterrupt: 